3.4

classification example ( movie review )

get data

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [ ]:
print(len(train_data),  len(train_labels), len(test_data), len(test_labels))
print()
print("max length of sequance: ", max([len(seq) for seq in train_data] ) )
print()
print("train data sampling:\n", train_data[0])
print()
print("label:", train_labels)
print()

# check max index of words
max([max(seq) for seq in train_data])

convert to original words (just test)

In [ ]:
word_index = imdb.get_word_index()
word_index

reverse_word_index = dict( [(value, key) for (key, value) in word_index.items() ] )
decoded_review = ' '.join( [reverse_word_index.get(i-3, '?') for i in train_data[0]] )
decoded_review



reday for data
- we cant inject numeric list to NN. so need to change list to tensor.
- there are two ways to make change.
- first method is `Embedding`
- second method is `One-hot encoding`
- in this section, we will use One-hot encoding

In [ ]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    # 크기가 (len(sequences), dimension))이고 모든 원소가 0인 행렬을 만듭니다
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # results[i]에서 특정 인덱스의 위치를 1로 만듭니다        
    return results

# 훈련 데이터를 벡터로 변환합니다
x_train = vectorize_sequences(train_data)
# 테스트 데이터를 벡터로 변환합니다
x_test = vectorize_sequences(test_data)

In [ ]:
x_train[0]

In [ ]:
# vectorize label

y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [ ]:
train_num = len(train_data)
test_num = len(train_data)

train_num

make NN model

In [ ]:
from keras import layers
from keras import models
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000, )) )
model.add(layers.Dense(16, activation='relu') )
model.add(layers.Dense(1, activation='sigmoid') )

prepare validation set

In [ ]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

network compile
- optimizer나 loss, metrics 등을 custom 가능함.

In [ ]:
from keras import losses
from keras import metrics

# use function
model.compile(optimizer=optimizers.RMSprop(),
                loss=losses.binary_crossentropy,
                metrics=[metrics.binary_accuracy]
               )

training network

In [ ]:
hist = model.fit(partial_x_train, partial_y_train, 
                   batch_size=512, 
                   epochs=20,
                   validation_data=(x_val, y_val)
                  )

result = model.evaluate(x_test, y_test)
result

check `history` object

In [ ]:
history_dict = hist.history
history_dict.keys()

draw cost graph (train, val)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

acc = hist.history['binary_accuracy']
val_acc = hist.history['val_binary_accuracy']
loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs = range(1, len(acc) + 1)

# ‘bo’는 파란색 점을 의미합니다
plt.plot(epochs, loss, 'bo', label='Training loss')
# ‘b’는 파란색 실선을 의미합니다
plt.plot(epochs, val_loss, 'b', label='Validation loss')

plt.plot(epochs, acc, 'ro', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')

plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


try make model again

In [ ]:
from keras import *

model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(10000, )) )
model.add(layers.Dense(32, activation='relu') )
model.add(layers.Dense(1, activation=activations.sigmoid) )

# use function
model.compile(optimizer=optimizers.RMSprop(),
                loss=losses.binary_crossentropy,
#                 loss=losses.mse,
                metrics=[metrics.binary_accuracy]
               )

# learning
tb_hist = callbacks.TensorBoard(log_dir='./graph', histogram_freq=0, write_graph=True, write_images=True)

hist = model.fit(partial_x_train, partial_y_train, 
                    batch_size=512, 
                    epochs=20,
                    validation_data=(x_val, y_val),
                     callbacks=[tb_hist]                 
                  )

evaluation

In [ ]:
result = model.evaluate(x_test, y_test)
result

prediction

In [ ]:
model.predict(x_test)

tensorboard 연동

In [ ]:
!pip install -q tensorboard

In [ ]:
%%bash --bg -s
nohup tensorboard --logdir="/home/boom109/workspace/keras-books-notebooks/soonhyung_notebooks/graph" 2>&1

In [ ]:
import webbrowser
webbrowser.open("http://0.0.0.0:6006")

In [ ]:
#실습
from keras import activations
from keras import regularizers
from keras import models
from keras import layers

m = models.Sequential()
m.add(layers.Dense(100, activation="relu", input_shape=(10000, )))
#m.add(layers.Dropout(0.5))
m.add(layers.Dense(50, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.01)))
m.add(layers.Dense(25, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.01)))
m.add(layers.Dense(1, activation=activations.tanh))
m.summary()      


In [ ]:
from keras import optimizers
from keras import losses
from keras import metrics
m.compile(optimizer=optimizers.RMSprop(lr=0.001), loss=losses.binary_crossentropy, metrics=[metrics.binary_accuracy])
m.fit(partial_x_train, partial_y_train, batch_size=512, epochs=30, validation_data=(x_val, y_val))

In [ ]:
!pip install -q tensorboard

In [ ]:
%%bash --bg -s
nohup tensorboard --logdir="/tf/notebooks/zo/soonhyung_notebooks/graph" 2>&1

In [ ]:
import webbrowser
webbrowser.open("http://0.0.0.0:6006")

In [ ]:
import pandas as pd
df = pd.DataFrame({"predict":m.predict(x_test), "label":y_test})